In [10]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import numpy as np
import hjson
import re
import matplotlib.pyplot as plt
from fuzzywuzzy import process
from unicodedata import normalize, combining
from datetime import datetime, timedelta

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from get_dataframes import GetDataframes
from tools import import_config, import_datasets, check_titre, color
from scipy.sparse import hstack

pd.set_option("display.max_columns", None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config = import_config()

In [8]:
acctor = pd.read_parquet("clean_datasets/site_web.parquet")

In [9]:
acctor

,budget,titre_genres,tmdb_id,titre_id,overview,popularity,production_countries,date,revenue,runtime,tagline,titre_str,rating_avg,rating_vote,keywords,production_companies_name,actors,director,url,image,youtube,titre_clean
0,400000,Comédie,8193,tt0374900,Napoleon Dynamite est un nerd de premier ordre...,16.76,[US],2004,46118097,94,Il veut prouver qu'il n'a rien à prouver.,Napoleon Dynamite,6.77,1696,"underdog, high school, sibling relationship, i...","[Access Films, Napoleon Pictures Limited]","Jon Heder, Efren Ramirez, Tina Majorino",Jared Hess,https://www.imdb.com/title/tt0374900,https://image.tmdb.org/t/p/w500/g5PAKdu5aolXMn...,https://www.youtube.com/watch?v=dQw4w9WgXcQ,napoleon dynamite
1,55000000,"Action, Thriller, Crime",8195,tt0122690,Cinq hommes et une femme qui ne se connaissent...,18.30,[US],1998,70692101,122,La trahison n'est qu'une question de prix.,Ronin,6.95,2154,"france, paris, france, arms deal, mercenary, s...","[United Artists, FGM Entertainment]","Robert De Niro, Jean Reno, Natascha McElhone",John Frankenheimer,https://www.imdb.com/title/tt0122690,https://image.tmdb.org/t/p/w500/yiZrJDZT8Yy8bf...,https://www.youtube.com/watch?v=vt0lr51NBFc,ronin
2,0,"Romance, Comédie, Drame",614409,tt10676012,"Tout a commencé avec une lettre d'amour, puis ...",24.97,[US],2021,0,109,On ne sait jamais où l'amour nous mènera,À tous les garçons : pour toujours et à jamais,7.59,1774,"based on novel or book, south korea, asian ame...","[Awesomeness Films, ACE Entertainment]","Lana Condor, Noah Centineo, Janel Parrish",Michael Fimognari,https://www.imdb.com/title/tt10676012,https://image.tmdb.org/t/p/w500/hEPXEchrF7aZ1U...,https://www.youtube.com/watch?v=ZtsGxoEq3V0,à tous les garçons : pour toujours et à jamais
3,11000000,"Aventure, Action, Science-Fiction",11,tt0076759,"Il y a bien longtemps, dans une galaxie très l...",101.31,[US],1977,775398007,121,Il y a bien longtemps dans une galaxie très lo...,La Guerre des étoiles,8.21,19321,"empire, galaxy, rebellion, android, hermit, sm...","[Lucasfilm Ltd., 20th Century Fox]","Mark Hamill, Harrison Ford, Carrie Fisher",George Lucas,https://www.imdb.com/title/tt0076759,https://image.tmdb.org/t/p/w500/qelTNHrBSYjPvw...,https://www.youtube.com/watch?v=HKQgMXvgxsM,la guerre des étoiles
4,94000000,"Animation, Familial",12,tt0266543,"Nemo, un jeune poisson-clown intrépide est cap...",96.28,[US],2003,940335536,101,71% de la surface du globe est constituée d'ea...,Le Monde de Nemo,7.83,18212,"sydney, australia, parent child relationship, ...",[Pixar],"Albert Brooks, Ellen DeGeneres, Alexander Gould",Andrew Stanton,https://www.imdb.com/title/tt0266543,https://image.tmdb.org/t/p/w500/8zR2vXoXfdlknE...,https://www.youtube.com/watch?v=TKi8VJmBuLs,le monde de nemo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3417,3500000,Drame,458737,tt6053438,"Toller, un ancien aumônier militaire, ravagé p...",23.28,"[AU, GB, US]",2018,2525738,108,,Sur le Chemin de la Rédemption,7.01,1228,"new york city, funeral, explosive, faith, chri...","[Arclight Films, Omeira Studio Partners, Kille...","Ethan Hawke, Amanda Seyfried, Cedric the Enter...",Paul Schrader,https://www.imdb.com/title/tt6053438,https://image.tmdb.org/t/p/w500/l2FIwSDeEAHBuG...,https://www.youtube.com/watch?v=M5WYaH9WMsA,sur le chemin de la rédemption
3418,0,"Drame, Thriller, Horreur",425972,tt3860916,Un père s'aventure dans la nature australienne...,19.95,"[AU, AE, GB]",2017,56385,104,,Cargo,6.41,1610,"australia, post-apocalyptic future, woman dire...","[Metrol Technology, Causeway Films, Head Gear ...","Martin Freeman, Simone Landers, Anthony Hayes","Ben Howling, Yolanda Ramke",https://www.imdb.com/title/tt3860916,https://image.tmdb.org/t/p/w500/cdPSUck4tBRvRu...,https://www.youtube.com/watch?v=Ahk8xruwKyk,cargo
3419,49000000,"Aventure, Comédie, Familial, Fantastique",499701,tt7547410,Après des années à explorer la jungle avec ses...,27.07,"[AU, US]",2019,120597108,102,Libérez votre côté sauvage.,Dora et

In [13]:
name = "clean_datasets/machine_learning_final.parquet"
df = import_datasets(name, "parquet")

2023-11-21 16:55:16 INFO     Parquet loaded ! Importing machine_learning_final...


In [14]:
df

,titre_id,titre_str,titre_genres,actors,actors_ids,director,director_ids,keywords,tmdb_id,overview,popularity,date,rating_avg,rating_vote,url,image,youtube
0,tt0374900,Napoleon Dynamite,comedie,"jonheder,efrenramirez,tinamajorino","[53926, 20190, 53930]",jaredhess,[53925],"underdog,highschool,siblingrelationship,idaho,...",8193,napoleondynamitenerdpremierordretendancenolife...,16.52,2004,6.77,1696,https://www.imdb.com/title/tt0374900,https://image.tmdb.org/t/p/w500/g5PAKdu5aolXMn...,https://www.youtube.com/watch?v=dQw4w9WgXcQ
1,tt0122690,Ronin,"action,thriller,crime","robertdeniro,jeanreno,nataschamcelhone","[380, 1003, 11317]",johnfrankenheimer,[13776],"france,paris,france,armsdeal,mercenary,suitcas...",8195,cinqhommesfemmeconnaissentrencontrententrepotb...,14.95,1998,6.95,2155,https://www.imdb.com/title/tt0122690,https://image.tmdb.org/t/p/w500/yiZrJDZT8Yy8bf...,https://www.youtube.com/watch?v=vt0lr51NBFc
2,tt10676012,À tous les garçons : pour toujours et à jamais,"romance,comedie,drame","lanacondor,noahcentineo,janelparrish","[1452046, 1253353, 93377]",michaelfimognari,[63099],"basedonnovelorbook,southkorea,asianamerican,se...",614409,toutacommencelettreamourpuispoursuivinouveller...,23.01,2021,7.59,1775,https://www.imdb.com/title/tt10676012,https://image.tmdb.org/t/p/w500/hEPXEchrF7aZ1U...,https://www.youtube.com/watch?v=ZtsGxoEq3V0
3,tt0076759,La Guerre des étoiles,"aventure,action,sciencefiction","markhamill,harrisonford,carriefisher","[2, 3, 4]",georgelucas,[1],"empire,galaxy,rebellion,android,hermit,smuggli...",11,abienlongtempsgalaxietreslointaineguerrecivile...,92.05,1977,8.20,19325,https://www.imdb.com/title/tt0076759,https://image.tmdb.org/t/p/w500/qelTNHrBSYjPvw...,https://www.youtube.com/watch?v=HKQgMXvgxsM
4,tt0266543,Le Monde de Nemo,"animation,familial","albertbrooks,ellendegeneres,alexandergould","[13, 14, 12]",andrewstanton,[7],"sydney,australia,parentchildrelationship,anthr...",12,nemojeunepoissonclownintrepidecapturepuisplong...,87.29,2003,7.83,18212,https://www.imdb.com/title/tt0266543,https://image.tmdb.org/t/p/w500/8zR2vXoXfdlknE...,https://www.youtube.com/watch?v=TKi8VJmBuLs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3418,tt6053438,Sur le Chemin de la Rédemption,drame,"ethanhawke,amandaseyfried,cedrictheentertainer","[569, 71070, 5726]",paulschrader,[1035],"newyorkcity,funeral,explosive,faith,christiani...",458737,tollerancienaumoniermilitaireravagemortfilscon...,17.75,2018,7.01,1228,https://www.imdb.com/title/tt6053438,https://image.tmdb.org/t/p/w500/l2FIwSDeEAHBuG...,https://www.youtube.com/watch?v=M5WYaH9WMsA
3419,tt3860916,Cargo,"drame,thriller,horreur","martinfreeman,simonelanders,anthonyhayes","[7060, 2045329, 75711]","benhowling,yolandaramke","[1167036, 1167037]","australia,postapocalypticfuture,womandirector,...",425972,pereaventurenatureaustralienneespoirtrouvergar...,18.95,2017,6.41,1611,https://www.imdb.com/title/tt3860916,https://image.tmdb.org/t/p/w500/cdPSUck4tBRvRu...,https://www.youtube.com/watch?v=Ahk8xruwKyk
3420,tt7547410,Dora et la cité perdue,"aventure,comedie,familial,fantastique","isabelamerced,jeffreywahlberg,madeleinemadden","[1428070, 1826682, 1109111]",jamesbobin,[26205],"peru,museum,fox,basedoncartoon,jungle,lostcity...",499701,apresanneesaexplorerjungleparentdoraprepareavi...,26.07,2019,6.67,1590,https://www.imdb.com/title/tt7547410,https://image.tmdb.org/t/p/w500/ycZjydbSh0U737...,https://www.youtube.com/watch?v=M1xZOvsd4Xo
3421,tt1482459,Le Lorax,"animation,familial","dannydevito,edhelms,zacefron","[518, 27105, 29222]",chrisrenaud,[124748],"aftermath,tree,familybusiness,money,tragicvill...",73723,conquerirurjolievoisineaudreytedvoixkevadamvae...,107.89,2012,6.49,3270,https://www.imdb.com/title/tt1482459,https://image.tmdb.org/t/p/w500/9pSXuNIgMnpgLp...,https://www.youtube.com/watch?v=9UooSj741GA


In [4]:
def titre_index(titre: str):
    return df[df.titre_str == titre].index[0]


def director_index(director: str):
    return df[df.directors.str.contains(director)].index[0]


def actor_index(actor: str):
    return df[df.actors.str.contains(actor)].index[0]


def idx_titre(idx: int):
    return df[df.index == idx]["titre_str"].values[0]


def idx_actor(idx: int):
    return df[df.index == idx]["actors"].values[0]


def idx_titre_id(idx: int):
    return df[df.index == idx]["titre_id"].values[0]


def idx_popularity(idx: int):
    return df[df.index == idx]["popularity"].values[0]


def idx_keywords(idx: int):
    return df[df.index == idx]["keywords"].values[0]


def idx_image(idx: int):
    return df[df.index == idx]["image"].values[0]


def idx_youtube(idx: int):
    return df[df.index == idx]["youtube"].values[0]


def idx_url(idx: int):
    return df[df.index == idx]["url"].values[0]


def check_titre_str(d: pd.DataFrame, movie: str):
    return df[df["titre_str"].str.contains(movie)]

In [6]:
check_titre(df, "ring")

KeyError: 'titre_clean'

In [ ]:
def combine(r):
    return (
        r["keywords"]
        + " "
        + r["actors"]
        + " "
        + r["director"]
        + " "
        +
        # r["overview"]+" "+
        r["titre_genres"]
        # str(r["date_only"])
        # str(r["popularity"])
        # str(r["rating_avg"])+ " "+
        # str(r["rating_vote"])
    )


df["one_for_all"] = df.apply(combine, axis=1)

In [ ]:
def get_best_match_index_tfidf(movies: str, df: pd.DataFrame):
    """
    Utilisation de FuzzyWuzzy et TfidfVectorizer
    """
    best_match = process.extract(
        movies, df["titre_clean"].values, limit=10
    )
    best_candidate = [match[0] for match in best_match]
    print("best_matches", best_match)
    print("best_candidate", best_candidate)
    print("movies", movies)

    small_df = df[df["titre_clean"].isin(best_candidate)]
    print(
        small_df[
            ["titre_id", "titre_str", "titre_genres", "keywords"]
        ].to_markdown()
    )
    print()

    tfidf = TfidfVectorizer()
    matrix = tfidf.fit_transform(best_candidate)
    tfidf_ = tfidf.transform([movies])

    cosine_similarities = cosine_similarity(tfidf_, matrix).flatten()

    best_match_idx = cosine_similarities.argmax()
    best_match_titre = best_candidate[best_match_idx]
    print("best_match_idx", best_match_idx)
    print("best_match_titre", best_match_titre)
    return df[df["titre_clean"] == best_match_titre].index[0]

In [ ]:
def get_best_match_index_knn(movies: str, df: pd.DataFrame):
    """
    Utilisation de FuzzyWuzzy et Nearest Neighbors
    """
    # best_match = process.extract(movies, df['titre_clean'].values, limit=10)
    best_match = process.extractOne(movies, df["titre_clean"].values)
    # best_candidate = [match[0] for match in best_match]
    # print("best_matches :\n",best_match[0])
    # print("best_candidate :\n",best_candidate)
    small_df = df[df["titre_clean"] == best_match[0]]
    # print(small_df[["titre_id", "titre_str", "titre_genres", "keywords"]].to_markdown())
    # small_df = df[df['titre_clean'].isin(best_candidate)]
    return df[df["titre_clean"] == best_match[0]].index[0]
    print(
        small_df[
            ["titre_id", "titre_str", "titre_genres", "keywords"]
        ].to_markdown()
    )
    print()

    tfidf = TfidfVectorizer()
    matrix = tfidf.fit_transform(small_df["titre_clean"].values)

    knn = NearestNeighbors(n_neighbors=1).fit(matrix)
    vector = tfidf.transform([movies])

    print("query_vector\n", vector)

    dist, idx = knn.kneighbors(vector, return_distance=True)

    best_match_idx = idx[0][0]
    best_match_titre = small_df["titre_clean"].iloc[best_match_idx]
    print("best_match_idx :", best_match_idx)
    print("best_match_titre :", best_match_titre)
    print()
    return small_df[small_df["titre_clean"] == best_match_titre].index[0]

In [ ]:
def get_best_match_index_rf(movies: str, df: pd.DataFrame):
    # Je capte pas le fonctionnement, j'ai besoin de plus de recherche
    raise NotImplementedError
    """
    Utilisation de FuzzyWuzzy et RandomForestClassifier
    """

    tfidf = TfidfVectorizer()
    X = tfidf.fit_transform(df["titre_clean"].values)
    y = df["titre_clean"].values

    y_encoded = LabelEncoder().fit_transform(y)
    rf = RandomForestClassifier().fit(X, y_encoded)

    vector = tfidf.transform([movies])
    prediction = rf.predict(vector)

    predict = y_encoded.inverse_transform(prediction)[0]
    return df[df["titre_clean"] == predict].index[0]

In [ ]:
def pick_algo(movies: str, df: pd.DataFrame, algo: str = "tfidf"):
    movies = (
        movies.replace(" ", "")
        .replace("-", "")
        .replace("'", "")
        .replace(":", "")
        .lower()
    )
    if algo == "tfidf":
        return get_best_match_index_tfidf(movies, df)
    elif algo == "knn":
        return get_best_match_index_knn(movies, df)
    elif algo == "rf":
        return get_best_match_index_rf(movies, df)

In [ ]:
def tfidf_algo(
    df: pd.DataFrame, movies: str, top: int = 10, algo: str = "tfidf"
):
    poids_ = {
        "titre_genres": 0.2,
        "actors": 0.15,
        "director": 0.15,
        "overview": 0.2,
        "keywords": 0.3,
    }

    full_matrix = []
    for col, poids in poids_.items():
        tfidf_ = TfidfVectorizer()
        matrix_ = tfidf_.fit_transform(df[col]) * poids
        full_matrix.append(matrix_)

    combined_matrix = hstack(full_matrix)
    cosine = cosine_similarity(combined_matrix)
    mov_idx = pick_algo(movies, df, algo)

    similar = cosine[mov_idx]
    similar1 = list(enumerate(cosine[mov_idx]))

    sim_scores = sorted(similar1, key=lambda x: x[1], reverse=True)
    sim_mov_idx = similar.argsort()[::-1][1 : top + 1]

    same_movies = df.loc[sim_mov_idx, "titre_str"]

    sim_scores[1 : top + 1]
    score = [i[1] for i in sim_scores]

    # imdb_url = "https://www.imdb.com/title/"
    # tmdb_image = "https://image.tmdb.org/t/p/w500"
    poster = f"Poster : {idx_image(mov_idx)}\n"

    print(color("~" * len(poster), "red"))
    print(f"Top 10 similar movies to {idx_titre(mov_idx)} are :")
    print(f"Popularity {idx_popularity(mov_idx)}")
    print(f"IMdb link : {idx_url(mov_idx)}")
    print(f"Poster : {idx_image(mov_idx)}")
    print(f"Youtube : {idx_youtube(mov_idx)}")
    print(color("~" * len(poster), "red"))
    print()
    for movies_, idx in zip(same_movies, sim_mov_idx):
        cmt = (
            f"Movie : {idx_titre(idx)} | popularity {idx_popularity(idx)}\n"
            + f"IMdb link : {idx_url(idx)}\n"
            + f"Poster : {idx_image(idx)}\n"
            + f"Youtube : {idx_youtube(idx)}\n"
        )
        line = cmt.split("\n")
        print(cmt + color("-" * len(max(line, key=len)), "green"))

In [ ]:
def cv_algo(
    df: pd.DataFrame, movies: str, top: int = 5, algo: str = "knn"
):
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(df["one_for_all"])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    mov_idx = pick_algo(movies, df, algo)

    sim_scores = list(enumerate(cosine_sim[mov_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1 : top + 1]
    movie_indices = [i[0] for i in sim_scores]

    # imdb_url = "https://www.imdb.com/title/"
    # tmdb_image = "https://image.tmdb.org/t/p/w500"
    poster = f"Poster : {idx_image(mov_idx)}\n"

    print(color("~" * len(poster), "red"))
    print(f"Top 10 similar movies to {idx_titre(mov_idx)} are :")
    print(f"Popularity {idx_popularity(mov_idx)}")
    print(f"IMdb link : {idx_url(mov_idx)}")
    print(f"Poster : {idx_image(mov_idx)}")
    print(f"Youtube : {idx_youtube(mov_idx)}")
    print(color("~" * len(poster), "red"))
    print()
    for idx in movie_indices:
        cmt = (
            f"Movie : {idx_titre(idx)} | popularity {idx_popularity(idx)}\n"
            + f"IMdb link : {idx_url(idx)}\n"
            + f"Poster : {idx_image(idx)}\n"
            + f"Youtube : {idx_youtube(idx)}\n"
        )
        line = cmt.split("\n")
        print(cmt + color("-" * len(max(line, key=len)), "green"))

In [ ]:
def knn_algo(
    df: pd.DataFrame, movies: str, top: int = 5, algo: str = "knn"
):
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(df["one_for_all"])
    mov_idx = pick_algo(movies, df, algo)

    knn_model = NearestNeighbors(metric="cosine", algorithm="brute").fit(
        count_matrix
    )
    dist, indices = knn_model.kneighbors(
        count_matrix[mov_idx], n_neighbors=top + 1
    )

    poster = f"Poster : {idx_image(mov_idx)}\n"
    print(color("~" * len(poster), "red"))
    print(f"Top 10 similar movies to {idx_titre(mov_idx)} are :")
    print(f"Popularity {idx_popularity(mov_idx)}")
    print(f"IMdb link : {idx_url(mov_idx)}")
    print(f"Poster : {idx_image(mov_idx)}")
    print(f"Youtube : {idx_youtube(mov_idx)}")
    print(color("~" * len(poster), "red"))
    print()
    for idx, dis in zip(indices.flatten()[1:], dist.flatten()[1:]):
        cmt = (
            f"Movie : {idx_titre(idx)} | popularity {idx_popularity(idx)}\n"
            + f"IMdb link : {idx_url(idx)}\n"
            + f"Poster : {idx_image(idx)}\n"
            + f"Youtube : {idx_youtube(idx)}\n"
        )
        line = cmt.split("\n")
        print(cmt + color("-" * len(max(line, key=len)), "green"))

AUTOPLAY : 
https://www.youtube.com/embed/MJ3Up7By5cw?autoplay=1&autohide=2&border=0&wmode=opaque&enablejsapi=1&modestbranding=1&controls=0&showinfo=1&mute=1

In [ ]:
movies = "fight club"

In [ ]:
tfidf_algo(df, movies, algo="knn", top=5)

In [ ]:
knn_algo(df, movies, algo="knn", top=5)

In [ ]:
cv_algo(df, movies, algo="knn", top=5)

In [ ]:
# random_forest_algo(df, movies, top=5)